## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-04-08-01 +0000,nyt,Why Israel Is Divided Over How to Investigate ...,https://www.nytimes.com/2026/01/06/world/middl...
1,2026-01-07-04-06-51 +0000,bbc,Chris Mason: Greenland and Ukraine point to Tr...,https://www.bbc.com/news/articles/cpqyleye482o...
2,2026-01-07-04-05-35 +0000,nyt,Live Updates: Trump Says Venezuela Will Start ...,https://www.nytimes.com/live/2026/01/06/world/...
3,2026-01-07-04-04-46 +0000,nypost,DOJ surging prosecutors to Minnesota to bring ...,https://nypost.com/2026/01/06/us-news/doj-surg...
4,2026-01-07-03-54-01 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/06/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2486/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,58
16,venezuela,50
84,maduro,23
20,oil,20
70,new,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
124,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...,176
61,2026-01-07-00-24-38 +0000,bbc,Trump says Venezuela will be 'turning over' u...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,153
54,2026-01-07-00-32-44 +0000,nypost,Venezuela selling up to 50 million barrels of ...,https://nypost.com/2026/01/06/us-news/venezuel...,151
39,2026-01-07-01-24-00 +0000,wsj,Venezuela to Give U.S. Up to 50 Million Barrel...,https://www.wsj.com/politics/policy/venezuela-...,151
2,2026-01-07-04-05-35 +0000,nyt,Live Updates: Trump Says Venezuela Will Start ...,https://www.nytimes.com/live/2026/01/06/world/...,143


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
124,176,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...
281,101,2026-01-06-11-30-00 +0000,nypost,Why fixing Venezuela’s oil industry is the onl...,https://nypost.com/2026/01/06/world-news/venez...
212,61,2026-01-06-17-47-55 +0000,nypost,AI deepfakes of Nicolás Maduro flood social me...,https://nypost.com/2026/01/06/business/ai-deep...
143,55,2026-01-06-21-51-00 +0000,wsj,Secretary of State Rubio told lawmakers that r...,https://www.wsj.com/politics/national-security...
163,51,2026-01-06-20-45-00 +0000,wsj,U.K. and France Agree to Set Up Military Hubs ...,https://www.wsj.com/world/europe/u-k-and-franc...
279,43,2026-01-06-11-53-09 +0000,nypost,Venezuelan opposition leader declares she woul...,https://nypost.com/2026/01/06/world-news/venez...
234,41,2026-01-06-16-16-08 +0000,nypost,Security clash with protesters in Iran’s Grand...,https://nypost.com/2026/01/06/world-news/secur...
182,39,2026-01-06-20-00-39 +0000,nypost,Gavin Newsom roasted for glowing praise of Ti...,https://nypost.com/2026/01/06/us-news/gavin-ne...
203,38,2026-01-06-18-36-00 +0000,wsj,"Republican Rep. Doug LaMalfa, who represented ...",https://www.wsj.com/politics/policy/doug-lamal...
65,33,2026-01-07-00-15-14 +0000,nypost,Tearful 911 call reported domestic dispute at ...,https://nypost.com/2026/01/06/us-news/911-call...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
